In [6]:
!pip install PyDictionary

usage: ssh [-1246AaCfGgKkMNnqsTtVvXxYy] [-b bind_address] [-c cipher_spec]
           [-D [bind_address:]port] [-E log_file] [-e escape_char]
           [-F configfile] [-I pkcs11] [-i identity_file]
           [-J [user@]host[:port]] [-L address] [-l login_name] [-m mac_spec]
           [-O ctl_cmd] [-o option] [-p port] [-Q query_option] [-R address]
           [-S ctl_path] [-W host:port] [-w local_tun[:remote_tun]]
           [user@]hostname [command]


# Explanation
First we need to install TensorFlow in the machine. Then `import` them.


In [0]:
import requests
from contextlib import closing
import codecs
import csv
import pandas as pd
import numpy as np
import pickle
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from sklearn.preprocessing import LabelBinarizer
import sklearn.datasets as skds
from pathlib import Path
from PyDictionary import PyDictionary
pydict = PyDictionary()

# Reproducability :D 
np.random.seed(7)

randomTextURL = 'https://raw.githubusercontent.com/carykh/neuralNetworkLanguageDetection/master/data/output0.txt'
keyMashTextURL = 'https://raw.githubusercontent.com/carykh/neuralNetworkLanguageDetection/master/data/output1.txt'
#bigNotGibberishURL = 'https://raw.githubusercontent.com/rrenaud/Gibberish-Detector/master/big.txt'

randomURLResponse = requests.get(randomTextURL)
keyMashURLResponse = requests.get(keyMashTextURL)
#bigNotGibberishURLResponse = request.urlopen(bigNotGibberishURL)

randomText = csv.reader(randomURLResponse.text)
keyMashText = csv.reader(keyMashURLResponse.text)

In [35]:
data_words, data_labels = [], []

for row in randomText:
  data_words.append(row[0])
  data_labels.append('Random Text')
  
for row in keyMash:
  data_words.append(row[0])
  data_labels.append('Keyboard Mash')
  
#for word in bigNotGibberish:
#  pass # something
  
# lets take 80% data as training and remaining 20% for test.
train_size = int(len(data) * .8)
 
train_words = data_words[:train_size]
train_labels = data_labels[:train_size]

test_words = data_words[train_size:]
test_labels = data_labels[train_size:]
  
# 2 different types
num_labels = 2
vocab_size = 15000
batch_size = 100
 
# define Tokenizer with Vocab Size
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(train_posts)
 
x_train = tokenizer.texts_to_matrix(train_words, mode='tfidf')
x_test = tokenizer.texts_to_matrix(test_words, mode='tfidf')
 
encoder = LabelBinarizer()
encoder.fit(train_tags)
y_train = encoder.transform(train_labels)
y_test = encoder.transform(test_labels)

['R']
['G']
['P']
['W']
['', '']
['1']
[]


IndexError: ignored

In [0]:
model = Sequential()
model.add(Dense(512, input_shape=(vocab_size,)))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(num_labels))
model.add(Activation('softmax'))
model.summary()
 
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
 
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=30,
                    verbose=1,
                    validation_split=0.1)

In [0]:
# Given a single, one-line string, determine if it contains an english word or
# not. If it does, determine whether this random text was produced via keyboard
# mashing or if it was randomly generated by a computer. Keyboard mashing likely
# contains letters around a certain key: if one key is pressed, as well as 6
# surrounding keys (at least 4), and this pattern appears for 75% of all of the
# random text, then it was probably mashed.

def determine_gibberish(line):
  gibberish = True
  gibberish_count = 0
  total_count = 0
  words = line.split()
  for word in words:
    if pydict.meaning(word) is None:
      gibberish_count+=1
      total_count+=1
    else:
      total_count+=1
      
  if gibberish_count / total_count < 0.25:
    gibberish = False
    
  return gibberish

def percent_gibberish(line):
  gibberish = True
  gibberish_count = 0
  total_count = 0
  words = line.split()
  for word in words:
    if pydict.meaning(word) is None:
      gibberish_count+=1
      total_count+=1
    else:
      total_count+=1
      
  return gibberish_count / total_count
  

In [0]:
def determine_mashing(list):
  # assumes list is gibberish
  # key set only includes the alphabet, not the other keys on the keyboard because i'm lazy
  common_keys = {"Q":["W","A"],"W":["Q","A","S","D","E"], "E":["W","S","D","F","R"], "R":["E","D","F","G","T"],"T":["R","F","G","H","Y"],
                "Y":["T","G","H","J","Y"], "U":["Y","H","J","I","K"], "I":["U","J","K","L","O"], "O":["I","K","L","P"],
                "A":["Q","W","S","Z"],"S":["Q","W","E","D","X","Z","A"],"D":["W","E","R","F","C","X","S"],"F":["D","E","R","T","G","V","C"],
                "G":["R","T","Y","H","B","V","F"],"H":["T","Y","U","J","N","B","G"],"J":["Y","U","I","K","M","N","H"],
                "K":["U","I","O","L","M","J"],"L":["I","O","P","K","M"],"Z":["A","S","X"],"X":["Z","S","D","C"],"C":["X","D","F","V"],
                "V":["C","F","G","B"],"B":["V","G","H","N"],"N":["B","H","J","M"],"M":["N","J","K","L",","]}
  
  mashing = False
  mash_count = 0
  words = line.split()
  # see moleskine notebook for the rest
  